# Nzalchemy load csv file using pandas and external table

### Setup

Start Jupyter notebook jupyter notebook

In this example, we will create a table 'sample' and try to load data file 'sample.csv' using pandas dataframe.

In [ ]:
from nzpy.core import NzTypeLastEntry
import sqlalchemy
import nzalchemy
import nzpy
import pandas
import os
import pdb
import csv
from sqlalchemy.sql.sqltypes import BIGINT, FLOAT

datapipe = 'datapipe1'

def exttbl_load(table, conn, keys, data_iter):
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        with open(datapipe, 'w', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerows(data_iter)
        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name
        sql = 'insert into {} select * from external \'{}\' using ( remotesource \'python\' delimiter \',\' quotedValue \'DOUBLE\' logDir \'C:\\Users\\ShabbirMohammad\\\')'.format(table_name, datapipe)
        cur.execute(sql)
        


### Load data from csv file

In [3]:
def creator():
    return nzpy.connect(user="admin", password="password",host='localhost', port=5480, database="db1", securityLevel=0,logOptions=nzpy.LogOptions.Logfile)
engine = sqlalchemy.create_engine("netezza+nzpy://", creator=creator)
print(engine)
data = pandas.read_csv('sample.csv')
data


Engine(netezza+nzpy://)


,Category,City,Country,Customer Name,Discount,Number of Records,Order Date,Order ID,Postal Code,Manufacturer,Product Name,Profit,Quantity,Region,Sales,Segment,Ship Date,Ship Mode,State,Sub-Category
0,Furniture,Henderson,United States,Claire Gute,0.00,1,11/8/2017,CA-2017-152156,42420.0,Bush,Bush Somerset Collection Bookcase,41.9136,2,South,261.9600,Consumer,11/11/2017,Second Class,Kentucky,Bookcases
1,Furniture,Henderson,United States,Claire Gute,0.00,1,11/8/2017,CA-2017-152156,42420.0,Hon,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",219.5820,3,South,731.9400,Consumer,11/11/2017,Second Class,Kentucky,Chairs
2,Office Supplies,Los Angeles,United States,Darrin Van Huff,0.00,1,6/12/2017,CA-2017-138688,90036.0,Universal,Self-Adhesive Address Labels for Typewriters b...,6.8714,2,West,14.6200,Corporate,6/16/2017,Second Class,California,Labels
3,Furniture,Fort Lauderdale,United States,Sean O'Donnell,0.45,1,10/11/2016,US-2016-108966,33311.0,Bretford,Bretford CR4500 Series Slim Rectangular Table,-383.0310,5,South,957.5775,Consumer,10/18/2016,Standard Class,Florida,Tables
4,Office Supplies,Fort Lauderdale,United States,Sean O'Donnell,0.20,1,10/11/2016,US-2016-108966,33311.0,Eldon,Eldon Fold 'N Roll Cart System,2.5164,2,South,22.3680,Consumer,10/18/2016,Standard Class,Florida,Storage
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,Technology,Los Angeles,United States,Ben Ferrer,0.20,1,12/24/2015,US-2015-138247,90045.0,Other,Seidio BD2-HK3IPH5-BK DILEX Case and Holster C...,17.3656,7,West,173.6560,Home Office,12/29/2015,Standard Class,California,Phones
9990,Technology,Los Angeles,United States,Ben Ferrer,0.20,1,7/12/2015,CA-2015-110184,90036.0,GE,GE DSL Phone Line Filter,28.7928,8,West,255.9360,Home Office,7/16/2015,Standard Class,California,Phones
9991,Technology,Santa Barbara,United States,Maria Etezadi,0.20,1,6/6/2017,CA-2017-160717,93101.0,Samsung,Samsung Galaxy Mega 6.3,226.7946,9,West,3023.9280,Home Office,6/11/2017,Standard Class,California,Phones
9992,Technology,Yuma,United States,Gary Hansen,0.70,1,10/16/2018,CA-2018-159282,85364.0,Other,Swingline SM12-08 MicroCut Jam Free Shredder,-479.9880,5,West,599.9850,Home Office,10/21/2018,Standard Class,Arizona,Machines


### Create table and load data using external table (by calling exttbl_load method)

In [4]:

my_types = {
    'Category': nzalchemy.CHAR(200),
    'City' : nzalchemy.CHAR(200),
    'Country' : nzalchemy.CHAR(200),
    'Customer Name' : nzalchemy.CHAR(200),
    'Discount' :      nzalchemy.FLOAT,
    'Number of Records' : nzalchemy.FLOAT,
    'Order Date' :   nzalchemy.CHAR(200),
    'Order ID' :     nzalchemy.CHAR(200),
    'Postal Code' :  nzalchemy.FLOAT,
    'Manufacturer' : nzalchemy.CHAR(200),
    'Product Name' : nzalchemy.CHAR(200),
    'Profit' :       nzalchemy.FLOAT,
    'Quantity' :     nzalchemy.BIGINT,
    'Region' :       nzalchemy.CHAR(200),
    'Sales' :        nzalchemy.FLOAT,
    'Segment' :      nzalchemy.CHAR(200),
    'Ship Date' :    nzalchemy.CHAR(200),
    'Ship Mode' :     nzalchemy.CHAR(200),
    'State' :        nzalchemy.CHAR(200),
    'Sub-Category' :  nzalchemy.CHAR(200)
}
data.to_sql('sample', engine, index=False, dtype=my_types, method=exttbl_load)
